In [1]:
from interface import UserInterface
from scipy.optimize import minimize
import alpaca_trade_api as tradeapi
import ipywidgets as widgets
import holoviews as hv
import hvplot.pandas
import pandas as pd
import numpy as np
import os

Exception: Section 'postgresql' not found in 'database.ini'.

In [ ]:
# Create an instance of the UserInterface class
interface = UserInterface()

 
## Login User
 

In [ ]:
user = interface.login_user()

 
## Select Portfolio
 

In [ ]:
portfolio = interface.select_portfolio()

## Get and Display Historical Data

In [ ]:
portfolio["historical_data"] = interface.get_historical_data(portfolio)
portfolio["historical_data"].head()

## Calculate and Display Beta Values (Portfolio)

In [ ]:
beta_values = interface.calculate_beta_values(portfolio)
interface.display_beta_values(beta_values)

## Setup and Run Monte Carlo Simulation (Portfolio)

In [ ]:
simulation = interface.create_simulation(portfolio, simulations=500, years=10)
simulation.calc_cumulative_return()

## Display Line Plot of Monte Carlo Simulation Results

In [ ]:
simulation.plot_simulation();

## *Simulated* Cumulative Returns for Portfolio

In [ ]:
portfolio["simulated_returns"] = simulation.summarize_cumulative_return()

## Display Cumulative Returns Summary

In [ ]:
print(portfolio["simulated_returns"])

## Display Initial Investment Forecast for Portfolio (Simulation)

In [ ]:
interface.display_portfolio_forecast(portfolio)


## If you don't like these simulated result, let's continue to see if we can do better...

### Lets see how you should invest your money!

In [ ]:
# Define variables needed below
start_date = pd.Timestamp("2017-11-01", tz="America/Phoenix").isoformat()
end_date = pd.Timestamp("2021-11-01", tz="America/Phoenix").isoformat()
amt_to_invest = portfolio["investment"]
tickers = portfolio["symbols"]
timeframe = "1D"

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")

if 'SPY' not in tickers:
    tickers.append('SPY')
    
# Function to get 3 years worth of historical data 
def api_get_barset(user_ticker, tm_frame, st_date, e_date):
    ticker_data = api.get_barset(
        user_ticker,
        tm_frame,
        start=st_date,
        end=e_date,
        limit=1000,
    ).df

    return(ticker_data)
    
def call_api(user_tickers):
    df_list = []

    for ticker in user_tickers:
        globals()[f"{ticker}_data"] = api_get_barset(ticker, timeframe, start_date, end_date)
        df_list.append(globals()[f"{ticker}_data"])
    
    return df_list

formatted_date_list = []
list_close_price_lists = []
stock_data_df_list = call_api(tickers)
date_list = stock_data_df_list[0].index.tolist()

for i, ticker in enumerate(tickers):
    globals()[f"{ticker}_closing_price_list"] = stock_data_df_list[i][ticker]['close'].to_list()
    list_close_price_lists.append(globals()[f"{ticker}_closing_price_list"])

for date in date_list:
    formatted_date_list.append(date.strftime("%Y-%m-%d"))

closing_price_dict = dict(zip(tickers, list_close_price_lists))        
closing_price_df = pd.DataFrame(closing_price_dict, index=formatted_date_list)
closing_price_df.index.name = 'Date'
closing_price_df.head()

Mean daily difference - Percentage change between the current and a prior element.

Computes the percentage change from the immediately previous row by default. This is useful in comparing the percentage of change in a time series of elements.

1 is default, meaning from one day to the next

In [ ]:
mean_daily_diff= closing_price_df.pct_change(1).mean()
mean_daily_diff

In [ ]:
closing_price_df.pct_change(1).corr()

Normalizing prices essentially allows you to view the comparison between the portfolio so that day one is effectively set to one for each member of the portfolio so the user is easily able to distinguish performance comparativley over time.

Note: need to account for stock splits somehow

In [ ]:
normalized_prices = closing_price_df/closing_price_df.iloc[0]
normalized_prices.hvplot()

In [ ]:
daily_returns = closing_price_df.pct_change(1)
daily_returns.dropna(inplace=True)
daily_returns.head()

Again, here, research shows that using logarithmic instead of pure returns is an advantageous method of normalizing the comparative items since the purpose of this tool is comparison and optimization.

In [ ]:
log_conversion = np.log(closing_price_df/closing_price_df.shift(1))
log_conversion.dropna(inplace=True)
log_conversion.head()

In [ ]:
log_conversion.hvplot.hist(bins=100, subplots=True, width=400, group_label='Ticker', grid=True).cols(2)

In [ ]:
log_conversion.describe().transpose()

In [ ]:
log_conversion.mean() * 252

In [ ]:
log_conversion.cov() * 252

Instead of setting your own weights let's generate random weights for our starting point in this analysis:

In [ ]:
weights = np.array(np.random.random(len(tickers)))

Then, set the weights so that they equal one (1.0) for the number of stocks the user is evaluating

In [ ]:
weights = weights / np.sum(weights)

Next, determine expected return for the random weights

In [ ]:
expected_returns = np.sum(log_conversion.mean() * weights) * 252

Next, search Google to find out how to calculate predicted volatility

In [ ]:
expected_volatility = np.sqrt(np.dot(weights.T, np.dot(log_conversion.cov() * 252, weights)))

Use expected returns and expected volatility to find Sharpe Ratio

In [ ]:
sharpe_ratio = expected_returns/expected_volatility

In [ ]:
investment_break_down = []
for weight, ticker in zip(weights, tickers):
    weighted_investment = float(amt_to_invest) * weight
    formatted_float = "{:.2f}".format(weighted_investment)
    print(f'For {ticker}, according to this randomly generated model, you should invest ${formatted_float}...')

Let's run 10000 simulations based on what we just did to see how this will look:

In [ ]:
simulations = 10000

Numpy.zeros takes an input (size) and fills it with zeros - in this case making new numpy arrays filled with zeros to be looped through

In [ ]:
new_weights = np.zeros((simulations,len(closing_price_df.columns)))
user_returns = np.zeros(simulations)
volatility = np.zeros(simulations)
new_sharpe_ratio = np.zeros(simulations)

In [ ]:
for sim in range(simulations):    
    weights = np.array(np.random.random(len(tickers)))
    weights = weights / np.sum(weights)
    new_weights[sim,:] = weights
    user_returns[sim] = np.sum((log_conversion.mean() * weights) *252)
    volatility[sim] = np.sqrt(np.dot(weights.T, np.dot(log_conversion.cov() * 252, weights)))
    new_sharpe_ratio[sim] = user_returns[sim]/volatility[sim]

Find the max sharpe ratio

In [ ]:
new_sharpe_ratio.max()

In [ ]:
sharpe_max_sim_count = new_sharpe_ratio.argmax()

Find the weights for the simulation in which the highest Sharpe Ratio occurred

In [ ]:
new_weights[sharpe_max_sim_count,:]

In [ ]:
max_weight_sim = new_weights[sharpe_max_sim_count,:]

In [ ]:
max_returns = user_returns[sharpe_max_sim_count]
max_return_volatility = volatility[sharpe_max_sim_count]

In [ ]:
print(max_returns)
print(max_return_volatility)

In [ ]:
scatter = hv.Scatter((volatility, user_returns, new_sharpe_ratio), 'Volatility', ['Return', 'Sharpe Ratio'])
max_sharpe = hv.Scatter([(max_return_volatility,max_returns)])

scatter.opts(color='Sharpe Ratio', cmap='plasma', width=600, height=400, colorbar=True, padding=0.1) *\
max_sharpe.opts(color='red', line_color='black', size=10)

In [ ]:
investment_break_down = []
for weight, ticker in zip(max_weight_sim, tickers):
    sim_weighted_investment = float(amt_to_invest) * weight
    formatted_float = "{:.2f}".format(sim_weighted_investment)
    print(f'For {ticker}, according to this model, you should invest ${formatted_float}...')

For the final component, we optimize the weights mathmathically using Scipy:

In [ ]:
# returns weighted Returns Volatility and Sharpe Ratio
def apply_weights(weights):
    weights = np.array(weights)
    rtn = np.sum(log_conversion.mean() * weights) * 252
    vola = np.sqrt(np.dot(weights.T, np.dot(log_conversion.cov() * 252, weights)))
    sharpe = rtn/vola
    return np.array([rtn,vola,sharpe])

https://en.wikipedia.org/wiki/Mathematical_optimization

In [ ]:
# minimize the negative Share Ratio
def minimized_sharpe(weights):
    return apply_weights(weights)[2] * -1

In [ ]:
# should equal zero assuming the weights are 1
def verify_weight(weights):
    return np.sum(weights) - 1

In [ ]:
# a minimizing function is a function returning 0, generally
conditions = ({'type':'eq','fun': verify_weight})

In [ ]:
# all bounds will be between 0 and 1

bounds = []

for x in range(len(tickers)):
    bounds.append((0,1))
bounds = tuple(bounds)

In [ ]:
# assume equal distribution in portfolio
distro = []
assumed_distro = 1/len(tickers)
for x in range(len(tickers)):
    distro.append(assumed_distro)
print(distro)

In [ ]:
# Sequential Least SUuares Programming (SLSQP)
optimization = minimize(minimized_sharpe,distro,method='SLSQP',bounds=bounds,constraints=conditions)

In [ ]:
frontier_y = np.linspace(0,0.3,100)

In [ ]:
def minimize_volatility(weights):
    return  apply_weights(weights)[1] 

In [ ]:
frontier_volatility = []

for possible_return in frontier_y:
    # function for return
    cons = ({'type':'eq','fun': verify_weight},
            {'type':'eq','fun': lambda w: apply_weights(w)[0] - possible_return})
    
    result = minimize(minimize_volatility,distro,method='SLSQP',bounds=bounds,constraints=conditions)
    
    frontier_volatility.append(result['fun'])

In [ ]:
scatter * hv.Curve((frontier_volatility, frontier_y)).opts(color='green', line_dash='dashed',title='Efficient Frontier')

In [ ]:
label_selected = widgets.Label(value="Selected: 0")
label_selected

In [ ]:
fig = log_conversion.hvplot.hist(bins=100, subplots=True, width=400, group_label='Ticker', grid=True).cols(2)
display(fig)

In [ ]:
fig = scatter * hv.Curve((frontier_volatility, frontier_y)).opts(color='green', line_dash='dashed')
display(fig)

In [ ]:
fig = scatter = hv.Scatter((volatility, user_returns, new_sharpe_ratio), 'Volatility', ['Return', 'Sharpe Ratio'])
max_sharpe = hv.Scatter([(max_return_volatility,max_returns)])
scatter.opts(color='Sharpe Ratio', cmap='plasma', width=600, height=400, colorbar=True, padding=0.1) *\
max_sharpe.opts(color='red', line_color='black', size=10)
display(fig)